In [6]:
import numpy as np
# import units as u
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt

from IPython.display import Markdown
from tabulate import tabulate

In [7]:
def print_nice(float):
    print(f"{float:,.0f}")
    return 
    
twh_to_mwh = 1e6
tj_to_mwh = 277.779
gwh_to_mwh = 1e3

# https://www.iea.org/countries/nigeria
elect_consumption = 27.6 # twh 
c_totalm = elect_consumption * twh_to_mwh
print_nice(c_totalm)

27,600,000


## Energy Consumption 

### Energy Consumption by Sector

In [8]:
tfc_sector = pd.read_csv("data/energy/tfc_sector.csv", skiprows=4) # TJ Total final consumption by sector 
tfc_sector

tc_sectorm = tfc_sector.loc[:, ['Industry', 'Transport', 'Residential',
       'Commercial and public services', 'Agriculture / forestry',
       'Non-energy use', 'Non-specified']] * tj_to_mwh
print_nice(tc_sectorm .iloc[-1].sum())
tc_sectorm

# sector consumption 


1,586,224,202


,Industry,Transport,Residential,Commercial and public services,Agriculture / forestry,Non-energy use,Non-specified
0,2.473066e+07,4.619548e+07,5.922296e+08,2.253954e+07,48333.546,4.100018e+06,NaN
1,2.811235e+07,5.842109e+07,6.703527e+08,2.096565e+07,60278.043,5.668358e+06,NaN
2,3.757655e+07,8.596566e+07,7.607220e+08,1.874453e+07,72222.540,3.577516e+06,4.453075e+06
3,6.460501e+07,1.134363e+08,8.707566e+08,1.607924e+07,120278.307,1.428979e+07,8.674760e+06
4,8.250703e+07,1.181294e+08,9.901694e+08,2.333816e+07,120556.086,7.779756e+06,1.534229e+07
5,8.527482e+07,1.771319e+08,1.126142e+09,3.813545e+07,48055.767,1.798063e+07,2.033065e+07
6,8.638371e+07,2.014978e+08,1.239870e+09,4.017101e+07,53055.789,1.622229e+07,2.026120e+06


In [9]:
fig = go.Figure(go.Pie(
    values = tc_sectorm.iloc[-1],
    labels = tc_sectorm.columns,
    pull=[0, 0, 0, 0, 0, 0, 0 ],
    # texttemplate = "%{label}: %{value:,s} <br>(%{percent})",
    # textposition = "outside"
    ))

fig.show()

In [10]:
# fig.write_image("figures/sector_cons.png")

### Energy Consumption by Source

In [11]:
tfc_source = pd.read_csv("data/energy/tfc_by_source.csv", skiprows=4) # TJ Total final consumption 
tfc_source

tc_sourcem = tfc_source.loc[:, ['Coal', 'Oil products', 'Natural gas',
       'Biofuels and waste', 'Electricity',]] * tj_to_mwh
print_nice(tc_sourcem .iloc[-1].sum())
tc_sourcem 


# total energy consumption by source 


1,586,224,479


,Coal,Oil products,Natural gas,Biofuels and waste,Electricity
0,449446.422,7.382671e+07,8.331426e+06,5.993646e+08,7.871146e+06
1,143333.964,8.389426e+07,8.608371e+06,6.814986e+08,9.436153e+06
2,21388.983,1.121033e+08,1.127116e+07,7.790723e+08,8.643094e+06
3,57222.474,1.416470e+08,3.272431e+07,8.963584e+08,1.717508e+07
4,272223.420,1.442676e+08,1.408784e+07,1.058099e+09,2.066009e+07
5,365557.164,2.115604e+08,4.376908e+07,1.184307e+09,2.504205e+07
6,311112.480,2.254965e+08,4.390880e+07,1.288741e+09,2.776707e+07


In [12]:
fig = go.Figure(go.Pie(
    values = tc_sourcem.iloc[-1],
    labels = tc_sourcem.columns,
    pull=[0, 0, 0, 0, 0.2],
    ))

fig.show()

### Calculate Energy Reduction 

In [13]:
perc_res = tc_sectorm.iloc[-1]["Residential"]/(tc_sectorm.iloc[-1].sum())
sources_2020 = tc_sourcem.iloc[-1]
res_sources = sources_2020*perc_res
res_sources

Coal                  2.431807e+05
Oil products          1.762591e+08
Natural gas           3.432126e+07
Biofuels and waste    1.007343e+09
Electricity           2.170409e+07
Name: 6, dtype: float64

In [14]:
mult_arr = [0.2, 0.2, 0.2, 0.2, 1] # Jacobson 7.3
mult = pd.DataFrame([(k,v) for k,v in zip(list(res_sources.index), mult_arr )])
mult.set_index(mult[0].values, drop=True, inplace=True)
mult 

,0,1
Coal,Coal,0.2
Oil products,Oil products,0.2
Natural gas,Natural gas,0.2
Biofuels and waste,Biofuels and waste,0.2
Electricity,Electricity,1.0


In [15]:
res_new = mult[1]* res_sources
res_new

Coal                  4.863614e+04
Oil products          3.525182e+07
Natural gas           6.864252e+06
Biofuels and waste    2.014686e+08
Electricity           2.170409e+07
dtype: float64

In [16]:
res_new.sum()

265337353.4671163

In [17]:
energy_dif = pd.DataFrame([res_sources, res_new], index=["BAU","WWS"])
energy_dif["Total"] = energy_dif.sum(axis=1)
energy_dif

,Coal,Oil products,Natural gas,Biofuels and waste,Electricity,Total
BAU,243180.683699,1.762591e+08,3.432126e+07,1.007343e+09,2.170409e+07,1.239870e+09
WWS,48636.136740,3.525182e+07,6.864252e+06,2.014686e+08,2.170409e+07,2.653374e+08


In [18]:
Markdown(energy_dif.to_markdown())

# 2.65e8 MW needed ...

|     |     Coal |   Oil products |   Natural gas |   Biofuels and waste |   Electricity |       Total |
|:----|---------:|---------------:|--------------:|---------------------:|--------------:|------------:|
| BAU | 243181   |    1.76259e+08 |   3.43213e+07 |          1.00734e+09 |   2.17041e+07 | 1.23987e+09 |
| WWS |  48636.1 |    3.52518e+07 |   6.86425e+06 |          2.01469e+08 |   2.17041e+07 | 2.65337e+08 |

In [19]:
labels = list(res_new.index)

fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['BAU', 'WWS'])
fig.add_trace(go.Pie(labels=labels, values=res_sources, scalegroup='one',
                     name="Energy Consumption by Source BAU"), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=res_new, scalegroup='one',
                     name="Energy Consumption by Source WWS"), 1, 2)

# fig.update_layout(title_text='Energy Consumption by Source BAU')
fig.show()

In [20]:
# fig.write_image("figures/source_change.png")

## Electricity Graphs 

In [21]:
consump_sector = pd.read_csv("data/energy/consumption_by_sector.csv", skiprows=4) # TJ 
print(consump_sector.head(1))
c_sector = consump_sector.loc[:, ["Industry", "Residential", "Commercial and public services"]]
c_sectorm = c_sector*tj_to_mwh
c_sectorm

# electricity consumption by sector 


   Unnamed: 0  Industry  Residential  Commercial and public services Units
0        1990      7258        14216                            6862    TJ


,Industry,Residential,Commercial and public services
0,2016119.982,3.948906e+06,1906119.498
1,2036953.407,4.950022e+06,2448899.664
2,1866119.322,4.431131e+06,2346121.434
3,2118898.212,1.030199e+07,4753909.806
4,3248903.184,1.196200e+07,5448912.864
5,3998073.147,1.437895e+07,6665029.326
6,3818627.913,1.636396e+07,7584477.816


In [24]:
print_nice(c_sectorm.iloc[-1] .sum())

27,767,067


In [26]:
gen_source = pd.read_csv("data/energy/generation_by_source.csv", skiprows=4) # GWh 
print(gen_source.head(1))
g_source = gen_source.loc[:, ["Coal", "Oil", "Natural gas", "Hydro", "Solar PV"]]
g_sourcem = g_source*gwh_to_mwh 
g_sourcem

# sorces of electricity generation 

   Unnamed: 0  Coal     Oil  Natural gas  Hydro  Solar PV Units
0        1990  13.0  1840.0         7223   4387       NaN   GWh


,Coal,Oil,Natural gas,Hydro,Solar PV
0,13000.0,1840000.0,7223000.0,4387000.0,NaN
1,NaN,3026000.0,7331000.0,5500000.0,NaN
2,NaN,NaN,9099000.0,5628000.0,NaN
3,NaN,NaN,15771000.0,7768000.0,NaN
4,NaN,NaN,19747000.0,6374000.0,NaN
5,NaN,NaN,26697000.0,6420000.0,25000.0
6,NaN,NaN,24701000.0,7686000.0,42000.0


In [27]:
print_nice(g_sourcem.iloc[-1] .sum())

32,429,000


In [105]:
fig = go.Figure(go.Pie(
   values = g_sourcem.iloc[-1],
    labels = g_sourcem.columns,
    # pull=[0, 0, 0, 0, 0.2],
    texttemplate = "%{label}: %{value:,s} <br>(%{percent})",
    textposition = "outside"
    ))

fig.show()

# Sources of electricity generation 

In [106]:
fig = go.Figure(go.Pie(
    values = c_sectorm.iloc[-1],
    labels = c_sectorm.columns,
    # pull=[0, 0, 0, 0, 0.2],
    texttemplate = "%{label}: %{value:,s} <br>(%{percent})",
    textposition = "outside"
    ))

fig.show()

# Sectoral split for electricity consumption 

In [79]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(
    values = g_sourcem.iloc[0],
    labels = g_sourcem.columns,),
    # texttemplate = "%{label}: %{value:,s} <br>(%{percent})",
    # textposition = "outside"),
    1,1)

fig.add_trace(go.Pie(
    values = c_sectorm.iloc[0],
    labels = c_sectorm.columns,
    # texttemplate = "%{label}: %{value:,s} <br>(%{percent})",
    # textposition = "outside"),
),
    1,2)

fig.show()